In [21]:
import java.nio.file.Files
import org.apache.spark.sql.SparkSession
//import spark.implicits._
//import org.apache.spark.sql.functions._

val spark = SparkSession
      .builder()
      .master("local[*]")
      .appName("Spark SQL KeepCoding Base")
      .getOrCreate()

import java.nio.file.Files
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4ce65384


In [22]:
// 1. Leemos ambos datasources cada uno con su formato correspondiente.
val csvDF = spark
      .read
      .format("csv")
      .option("sep", ",")
      .option("inferSchema", "true")
      .option("header", "true")
      .load("Data/sample.csv")

val jsonDF = spark
      .read
      .format("json")
      .load("Data/sample.json")

csvDF: org.apache.spark.sql.DataFrame = [nombre: string, apellido: string ... 2 more fields]
jsonDF: org.apache.spark.sql.DataFrame = [apellido: string, clase: string ... 2 more fields]


In [23]:
// 2. Unificamos los datos CSV & JSON, usando la función unionByName(...)
val fullDF = jsonDF.unionByName(csvDF).cache()

fullDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [apellido: string, clase: string ... 2 more fields]


In [24]:
// 3. ¿Qué alumno tiene mejor nota media del instituto?

fullDF
      .select($"nombre", $"apellido", $"nota")
      .groupBy($"nombre", $"apellido")
      .agg(avg($"nota").alias("nota_media"))
      .sort($"nota_media".desc)
      .limit(1)
      .show()

+------+--------+----------+
|nombre|apellido|nota_media|
+------+--------+----------+
|  sara|  garcia|       9.0|
+------+--------+----------+



In [25]:
// 4. ¿Qué clase tiene las peores notas?

fullDF
      .select($"clase", $"nota")
      .groupBy($"clase")
      .agg(avg($"nota").as("nota_media"))
      .sort($"nota_media".asc)
      .limit(1)
      .show()

+------+----------+
| clase|nota_media|
+------+----------+
|ingles|       6.6|
+------+----------+



In [26]:
// 5. Añade el tipo de curso a cada alumno 'ciencias'/'humanidades'
    //    dentro de la columna 'curso', escribe los resultados en formato JSON
    //    en un directorio temporal:
    //
    //    val outputDir = s"""${Files.createTempDirectory("spark").toFile.getAbsolutePath}/results""

val cursoDF = fullDF
      .select($"nombre", $"apellido", $"clase")
      .groupBy($"nombre", $"apellido")
      .agg(collect_list($"clase").as("clases"))
      .withColumn("curso",
        when(
          $"clases" === array(Seq("matematicas", "tecnologia", "ingles").map(lit): _*),
          "ciencias"
        ).otherwise("humanidades")
      )
      .select($"nombre", $"apellido", $"curso")

    val resultDF = fullDF.as("a")
      .join(cursoDF.as("b"), $"a.nombre" === $"b.nombre" && $"a.apellido" === $"b.apellido")
      .select((fullDF.columns.map(name => col(s"a.$name").as(name)) :+ col("b.curso").as("curso")): _*)
      .cache()

    //val outputDir = s"""${Files.createTempDirectory("spark").toFile.getAbsolutePath}/results"""
    val outputDir = "Resultados/exercise2_output"
    resultDF.show()
    resultDF
      .write
      .format("json")
      .save(outputDir)

    println(s"Result write into $outputDir")
 //   spark.close()

+--------+-----------+------+----+-----------+
|apellido|      clase|nombre|nota|      curso|
+--------+-----------+------+----+-----------+
|  ferrer|   historia|  juan|  10|humanidades|
|  ferrer|  filosofia|  juan|   8|humanidades|
|  ferrer|     ingles|  juan|   7|humanidades|
|  garcia|matematicas|  sara|   9|   ciencias|
|  garcia| tecnologia|  sara|  10|   ciencias|
|   gomez|   historia|  juan|   6|humanidades|
|   gomez|  filosofia|  juan|   8|humanidades|
|   gomez|     ingles|  juan|   5|humanidades|
|  garcia|     ingles|  sara|   8|   ciencias|
|   gomez|matematicas|  pepe|   8|   ciencias|
|   gomez| tecnologia|  pepe|   7|   ciencias|
|   gomez|     ingles|  pepe|   7|   ciencias|
| fuentes|  filosofia| lucia|   7|humanidades|
| fuentes|   historia| lucia|   8|humanidades|
| fuentes|     ingles| lucia|   6|humanidades|
+--------+-----------+------+----+-----------+

Result write into Resultados/exercise2_output


cursoDF: org.apache.spark.sql.DataFrame = [nombre: string, apellido: string ... 1 more field]
resultDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [apellido: string, clase: string ... 3 more fields]
outputDir: String = Resultados/exercise2_output
